In [ ]:
import requests
import json
from api_key import api_key
import pandas as pd

In [189]:
class Stock():
    def __init__(self, symbol,api_key):
        self.symbol=symbol
        self.api_key=api_key
        self.api_output=self.get_api_data()
        self.weight=1
        
    def get_api_data(self):
        """make api call"""
        #Would want to handle non-200 responses 
        r=requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+self.symbol+'&outputsize=full&apikey='+self.api_key)
        return r
    
    def get_stock_df(self):
        """Create a dataframe indexed by day wwhich has share price info"""
        stock_hist=json.loads(self.api_output.content)
        stock_df=pd.DataFrame(stock_hist['Time Series (Daily)']).transpose()
        stock_df=stock_df.astype(float)
        stock_df['9. symbol']=stock_hist['Meta Data']['2. Symbol']
        return stock_df
    
    #This should probably be a class since we're going to be doing a bunch of things with the S&P
class Index():
    def __init__(self):
        self.sp500_df=self.get_sp_df()
        
    def get_sp_df(self):
        #Need a better source for SP500 financials like mkcp
        sp500=requests.get('https://datahub.io/core/s-and-p-500-companies-financials/r/constituents-financials.json')
        sp500_df=pd.DataFrame(json.loads(sp500.content))
        sp500_df['Shares']=sp500_df['Market Cap']/sp500_df['Price']
        return sp500_df
    def sample_sp(self,num_sample):
        return self.sp500_df.sample(num_sample)

    
    
class Portfolio():
    def __init__(self, symbol_list):
        self.symbol_list=symbol_list
        self.Stocks=self.get_all_stocks()
    def get_all_stocks(self):
        Stocks=[]
        for symbol in self.symbol_list:
            Stocks.append(Stock(symbol,api_key))
        return Stocks
    def total_portfolio_df(self):
        loop_check=0
        for stock in self.Stocks:
            if loop_check==0:
                agg_df=stock.weight*Stocks.get_stock_df()
            else:
                agg_df+=Stocks.get_stock_df()
        

In [121]:
#sp500=Index()
sp500.sample_sp(1)


,52 Week High,52 Week Low,Dividend Yield,EBITDA,Earnings/Share,Market Cap,Name,Price,Price/Book,Price/Earnings,Price/Sales,SEC Filings,Sector,Symbol,Shares
58,32.55,42.7,5.415651,4.965300e+10,4.76,2.267133e+11,AT&T Inc,35.57,1.8,12.14,1.408315,http://www.sec.gov/cgi-bin/browse-edgar?action...,Telecommunication Services,T,6.373721e+09


In [190]:
stock1=Stock('DIS',api_key)
stock2=Stock('T',api_key)
combined=stock1.get_stock_df().join(stock2.get_stock_df(),lsuffix=' '+stock1.symbol,rsuffix=' '+stock2.symbol)

In [191]:
stock1.get_stock_df()+stock2.get_stock_df()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,9. symbol
1998-01-02,173.8100,175.0000,172.3800,174.57,38.3817,3170133.0,0.000,2.0,DIST
1998-01-05,174.5000,174.8000,169.3200,171.62,37.8111,6917133.0,0.000,2.0,DIST
1998-01-06,171.0600,171.4400,170.0600,170.94,37.6632,4342400.0,0.000,2.0,DIST
1998-01-07,170.2500,171.4400,169.0700,171.44,37.8626,3510067.0,0.581,2.0,DIST
1998-01-08,170.3100,170.6900,168.1900,169.94,37.5360,2789300.0,0.000,2.0,DIST
1998-01-09,169.8200,170.6300,166.7500,167.56,36.9815,3447267.0,0.000,2.0,DIST
1998-01-12,165.0600,168.7600,164.5000,167.07,36.7588,3652067.0,0.000,2.0,DIST
1998-01-13,167.3800,168.8100,166.3100,168.31,37.0190,3010733.0,0.000,2.0,DIST
1998-01-14,168.4400,170.7600,167.3200,170.13,37.4616,2352667.0,0.000,2.0,DIST
1998-01-15,169.3800,170.3100,167.0000,169.63,37.3044,2368433.0,0.000,2.0,DIST


In [177]:
#combined

In [180]:
combined[['5. adjusted close '+stock1.symbol,'5. adjusted close '+stock2.symbol]].astype(float).corr()

,5. adjusted close SPY,5. adjusted close NFLX
5. adjusted close SPY,1.000000,0.903741
5. adjusted close NFLX,0.903741,1.000000


In [ ]:


stock_list=sp500_df.sample(5)['Symbol'].tolist()
index=Portfolio(stock_list)
index.Stocks[0].get_stock_df()

In [ ]:
z=Stock('DIS',api_key)
z.get_stock_df()